In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import auc, roc_curve
from category_encoders.target_encoder import TargetEncoder
# 导入数据
train = pd.read_csv('F://天池/贷款违约预测/train.csv', index_col='id')
test = pd.read_csv('F://天池/贷款违约预测/testA.csv', index_col='id')
target = train.pop('isDefault')
pd.set_option('display.max_columns', None)
train.head()
geshi=train.apply(lambda x:x.dtype)
geshi1=test.apply(lambda x:x.dtype)
ewai=train[geshi[geshi=='object'].index.tolist()]
ewai2=test[geshi1[geshi1=='object'].index.tolist()]




train=train.drop(geshi[geshi=='object'].index.tolist(),axis=1)
test=test.drop(geshi1[geshi1=='object'].index.tolist(),axis=1)
train[geshi[geshi=='int64'].index.tolist()]=train[geshi[geshi=='int64'].index.tolist()].astype(np.float64)
train.apply(lambda x:x.dtype)
train=pd.DataFrame(train)
test=pd.DataFrame(test)
test.head()
def makelgb():
    lgbr = LGBMRegressor(num_leaves=22
                        ,max_depth=30
                        ,learning_rate=0.15
                        ,n_estimators=300
                        ,subsample_for_bin=50000
                        ,min_child_samples=10
                        ,colsample_bytree=1
                        ,reg_alpha=.1
                        ,reg_lambda=.1
                        
                        )
    return lgbr

queshi_list=train.isnull().any().index.tolist()



for i in queshi_list:
    train[i].fillna(axis=0,method='bfill',limit=2)

    
queshi_list1=test.isnull().any().index.tolist()

for i in queshi_list1:
    test[i].fillna(axis=0,method='bfill',limit=2)
test.head()
tecols=geshi[geshi=='object'].index.tolist()
tecols

train=pd.concat([train,ewai],axis=1)
test=pd.concat([test,ewai2],axis=1)
train
from datetime import datetime
#转化成时间格式
train['issueDate'] = pd.to_datetime(train['issueDate'],format='%Y-%m-%d')
startdate = datetime.strptime('2007-06-01', '%Y-%m-%d')
train['issueDateDT'] = train['issueDate'].apply(lambda x: x-startdate).dt.days

#转化成时间格式
test['issueDate'] = pd.to_datetime(test['issueDate'],format='%Y-%m-%d')
startdate = datetime.strptime('2007-06-01', '%Y-%m-%d')
test['issueDateDT'] = test['issueDate'].apply(lambda x: x-startdate).dt.days

estimator=makelgb()
te = TargetEncoder(cols=tecols)

tf = te.fit_transform(train, target)

df = te.transform(test)
estimator.fit(tf,target)

test_predict=estimator.predict(df)
np.set_printoptions(threshold=np.inf)  
test_predict
pd.Series(test_predict, name='isDefault', index=test.index).reset_index().to_csv('submit1.csv', index=False)

# 本地验证
kf = KFold(n_splits=10, shuffle=True, random_state=100)
devscore = []
for tidx, didx in kf.split(train.index):
    tf = train.iloc[tidx]
    df = train.iloc[didx]
    tt = target.iloc[tidx]
    dt = target.iloc[didx]
    te = TargetEncoder(cols=tecols)
    tf = te.fit_transform(tf, tt)
    df = te.transform(df)
    lgbr = makelgb()
    lgbr.fit(tf, tt)
    pre = lgbr.predict(df)
    fpr, tpr, thresholds = roc_curve(dt, pre)
    score = auc(fpr, tpr)
    devscore.append(score)
print(np.mean(devscore))

d:\hwy\python\an\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
d:\hwy\python\an\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
d:\hwy\python\an\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
d:\hwy\python\an\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
d:\hwy\python\an\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype ins

0.7333188122633445
